# UHS4 imputation
__Author__: Jesse Marks <br>

**GitHub Issue:** [Issue #117](https://github.com/RTIInternational/bioinformatics/issues/117)

This document logs the steps taken to perform pre-imputation procedures on the UHS4 dataset, both EA and AA. The starting point for this analysis is after quality control of observed genotypes. The quality controlled genotypes are oriented on the GRCh37 plus strand. 

## Software and tools
The software and tools used for porcessing these data are
* [Michigan Imputation Server](https://imputationserver.sph.umich.edu/index.html) (MIS)
* [Amazon Web Services (AWS) - Cloud Computing Services](https://aws.amazon.com/)
    * Linux AMI
* [PLINK v1.90 beta 4.10](https://www.cog-genomics.org/plink/)
* [bgzip](http://www.htslib.org/doc/tabix.html)
* [BCF Tools](http://www.htslib.org/doc/bcftools.html)
* Windows 10 with [Cygwin](https://cygwin.com/) installed
* GNU bash version 4.2.46

## Data retrieval and organization
PLINK binary filesets will be obtained from AWS S3 storage.

Jesse Marks performed the QC and stored the observed genotype data at:

```
s3/rti-heroin/uhs4/data/genotype/observed/final/
```

###  Statistics Breakdown 
This table includes the initial number of variants in each study as well as the final number of variants in the intersection set. The `Variants Post-Filtering` is in referral to the filtering steps (1) remove discordant alleles & (2) removal of monomorphic variants.

#### EA
| Data Set      | Initial Variants | Variants Post-Filtering  | Intersection     |
|---------------|------------------|--------------------------|------------------|
| Kreek         |                  |                          | NA               |


#### AA
| Data Set      | Initial Variants | Variants Post-Filtering  | Intersection     |
|---------------|------------------|--------------------------|------------------|
| Kreek         |                  |                          | NA               |


## Create Directory Structure & Download Data
The following section needs to be modified each time to reflect where the data is stored!

In [1]:
### EC2 command line (bash) ###

# Modify variables below
######################################################################
base_dir=/shared/data/studies/heroin/uhs4/genotype/imputed/20181209
genD=/shared/data/studies/heroin/uhs4/genotype/observed/final

base_name="chr_all" # chr23 or chr_all
chr_list={1..23} # or {1..22} 
ancestry_list="aa ea" # space delimited Ex. "ea aa ha"
study_list="uhs4" # space delimited, lowercase
######################################################################

mkdir ${base_dir}/processing/{intersect,1000g,impute_prep}
for study in ${study_list};do
    for ancestry in ${ancestry_list};do
        mkdir -p ${base_dir}/processing/${study}
        mkdir -p ${base_dir}/data/${study}/genotype/observed/${ancestry}
    done
done

## download genotype (with AWS CLI tools) to respective directories ##
#aws s3 sync s3://rti-heroin/kreek/data/genotype/original/20181128/ \
#    ${base_dir}/data/${study}/genotype/observed/
#mv ${base_dir}/data/${study}/genotype/observed/ea* ${base_dir}/data/${study}/genotype/observed/ea
#mv ${base_dir}/data/${study}/genotype/observed/aa* ${base_dir}/data/${study}/genotype/observed/aa

# Data Processing
## GRCh37 strand and allele discordance check
### MAF for study data

In [ ]:
### EC1 command line (Bash) ###

# write out the MAF report
for study in ${study_list}; do
    study_dir=${base_dir}/processing/${study}/strand_check
    mkdir ${study_dir}
    for ancestry in ${ancestry_list};do
        data_dir=${base_dir}/data/${study}/genotype/observed/${ancestry}
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bed ${data_dir}/*bed\
            --bim ${data_dir}/*bim\
            --fam ${data_dir}/*fam\
            --freq \
            --out ${study_dir}/${ancestry}_${base_name}
    done
done

# Get list of variants from all studies
for ancestry in ${ancestry_list}; do
    for study in ${study_list};do
        cat ${base_dir}/data/${study}/genotype/observed/${ancestry}/*bim | \
                perl -lane 'if (($F[0]+0) <= 23) { print $F[1]; }' | \
                sort -u > ${base_dir}/processing/${ancestry}_${base_name}_sorted_variants.txt
    done
done

```
 wc -l ${base_dir}/processing/*
  946932 /shared/data/studies/heroin/uhs4/genotype/imputed/20181209/processing/aa_chr_all_sorted_variants.txt
 1154359 /shared/data/studies/heroin/uhs4/genotype/imputed/20181209/processing/ea_chr_all_sorted_variants.txt
```

### MAF for 1000G
This pipeline is currently set up to handle EUR and AFR populations.
#### Autosomes
Get 1000G MAF for chromosomes 1–22 (autosomes).

In [ ]:
### EC2 command line (Bash)

# Calculate autosome MAFs for 1000G populations
for ancestry in ${ancestry_list};do

    if [ $ancestry == "ea" ]
    then
        pop="EUR"
    else
        pop="AFR"
    fi
    
    for chr in {1..22}; do
        /shared/bioinformatics/software/scripts/qsub_job.sh \
            --job_name ${pop}_${chr}_MAF \
            --script_prefix ${base_dir}/processing/1000g/${pop}_chr${chr}.maf \
            --mem 6.8 \
            --nslots 1 \
            --priority 0 \
            --program /shared/bioinformatics/software/perl/stats/calculate_maf_from_impute2_hap_file.pl \
                --hap /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chr${chr}.hap.gz\
                --legend /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chr${chr}.legend.gz \
                --sample /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3.sample \
                --chr ${chr} \
                --out ${base_dir}/processing/1000g/${pop}_chr${chr}.maf \
                --extract ${base_dir}/processing/${ancestry}_${base_name}_sorted_variants.txt \
                --keep_groups ${pop}
    done
done

#### chrX
Get 1000G MAF for chromosome 23 (chrX).

In [ ]:
### Bash ###

chr=23
for ancestry in ${ancestry_list};do

    if [ $ancestry == "ea" ]
    then
        pop="EUR"
    else
        pop="AFR"
    fi

    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${pop}_23_MAF \
        --script_prefix ${base_dir}/processing/1000g/${pop}_chr${chr}.maf \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program /shared/bioinformatics/software/perl/stats/calculate_maf_from_impute2_hap_file.pl \
            --hap /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chrX_NONPAR.hap.gz\
            --legend /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chrX_NONPAR.legend.gz \
            --sample /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3.sample \
            --chr $chr \
            --out ${base_dir}/processing/1000g/${pop}_chr${chr}.maf \
            --extract ${base_dir}/processing/${ancestry}_${base_name}_sorted_variants.txt \
            --keep_groups ${pop}
done

### Merge 1000G chromosomes
Only need to perform this if there were multiple chromosomes for which the MAF was calculated—e.g. more than just chrX.

In [ ]:
### Bash ###

# Merge per chr MAFs for each 1000G population
for ancestry in ${ancestry_list};do
    if [ $ancestry == "ea" ]
    then
        pop="EUR"
    else
        pop="AFR"
    fi
    
        head -n 1 ${base_dir}/processing/1000g/${pop}_chr1.maf > ${base_dir}/processing/1000g/${pop}_chr_all.maf
        tail -q -n +2 ${base_dir}/processing/1000g/${pop}_chr??maf  >> ${base_dir}/processing/1000g/${pop}_chr_all.maf
done

```
wc -l processing/1000g/???_chr_all.maf
  520438 processing/1000g/AFR_chr_all.maf
  628281 processing/1000g/EUR_chr_all.maf
```

###  Allele Discordances Check
The allele discordances will be resolved by
* Flipping allele discordances that are fixed by flipping
* Removing SNPs with discordant names
* Removing SNPs with discordant positions
* Removing allele discordances that are not resolved by flipping
* Removing alleles with large deviations from the reference population allele frequencies

Given that the allele discordance check was done using a union set of SNPs across all studies within an ancestry group, some of the SNPs logged as discordant for a given study may not actually be in the study. Fortunately, if they are not in a given study they will not interfere with the filtering procedures. Note that the intersection set is used for the final studies merger.

#### Autosomes

In [ ]:
### Bash ###

# Run discordance checks for each ancestry group
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        if [ $ancestry = "ea" ]; then
            pop=EUR
        else
            pop=AFR
        fi

       /shared/bioinformatics/software/scripts/qsub_job.sh \
           --job_name ${ancestry}_${study}_crosscheck_chr$chr \
           --script_prefix ${base_dir}/processing/$study/strand_check/${ancestry}_allele_discordance_check \
           --mem 6 \
           --nslots 4 \
           --priority 0 \
           --program "Rscript /shared/bioinformatics/software/R/check_study_data_against_1000G.R
               --study_bim_file ${base_dir}/data/${study}/genotype/observed/${ancestry}/*bim
               --study_frq_file ${base_dir}/processing/${study}/strand_check/${ancestry}_chr_all.frq
               --ref_maf_file ${base_dir}/processing/1000g/${pop}_chr_all.maf
               --out_prefix ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance"
    done
done

#### chrX 
Not necessary to run unless you are only processing chrX.

In [ ]:
### Bash ###

for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        if [ $ancestry = "ea" ]; then
            pop=EUR
        else
            pop=AFR
        fi

        # chr23 discordance check
        /shared/bioinformatics/software/scripts/qsub_job.sh \
            --job_name ${ancestry}_${study}_crosscheck_chr$chr \
            --script_prefix ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance_check \
            --mem 6.8 \
            --nslots 1 \
            --priority 0 \
            --program "Rscript /shared/bioinformatics/software/R/check_study_data_against_1000G.R
                --study_bim_file ${base_dir}/data/${study}/genotype/observed/${ancestry}/*bim
                --study_frq_file ${base_dir}/processing/${study}/strand_check/${ancestry}_chr23.frq
                --ref_maf_file ${base_dir}/processing/1000g/${pop}_chr23.maf
                --out_prefix ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance"
    done
done

### Resolving Allele Discordances

In [ ]:
### Bash ###

# Apply filters
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        echo -e "\n===============\nProcessing ${study}_${ancestry}\n"
        echo "Making remove list"
        cat <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_not_fixed_by_strand_flip | tail -n +2) \
            <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.at_cg_snps_freq_diff_gt_0.2 | tail -n +2) \
            <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_names | tail -n +2) \
            <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_positions | tail -n +2) \
            <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_polymorphic_in_study_not_fixed_by_strand_flip | tail -n +2) | \
              sort -u > ${base_dir}/processing/${study}/strand_check/${ancestry}_snps.remove

        # Create flip list
        echo "Making flip list"
        comm -23 <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles | tail -n +2 | sort -u) \
                 <(cut -f2,2 ${base_dir}/processing/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_not_fixed_by_strand_flip | tail -n +2 | sort -u) \
                 > ${base_dir}/processing/${study}/strand_check/${ancestry}_snps.flip

        # Apply filters
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bed     ${base_dir}/data/${study}/genotype/observed/${ancestry}/*bed \
            --bim     ${base_dir}/data/${study}/genotype/observed/${ancestry}/*bim \
            --fam     ${base_dir}/data/${study}/genotype/observed/${ancestry}/*fam \
            --exclude ${base_dir}/processing/${study}/strand_check/${ancestry}_snps.remove \
            --flip    ${base_dir}/processing/${study}/strand_check/${ancestry}_snps.flip \
            --make-bed \
            --out     ${base_dir}/processing/${study}/${ancestry}_filtered
    done
done

 ```
 wc -l processing/*/*bim
  945981 processing/uhs4/aa_filtered.bim
 1153403 processing/uhs4/ea_filtered.bim
 ```

## Remove monomorphic variants
Monomorphic variants prevent MIS from accepting the genotype data. In this case, an arbitrarily small MAF is set that is smaller than the lower bound for these data.

In [ ]:
### Bash ###

# Apply filters
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        geno_dir=${base_dir}/processing/${study}

        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bfile ${geno_dir}/${ancestry}_filtered \
            --maf 0.000001 \
            --make-bed \
            --out ${geno_dir}/${ancestry}_filtered_mono
    done
done

```
wc -l processing/*/*mono.bim

  878900 processing/uhs4/aa_filtered_mono.bim
  882603 processing/uhs4/ea_filtered_mono.bim
  ```

### SNP intersection
Only run if merging multiple data sets.

In [ ]:
#studies=($study_list)  #studies=(UHS1 UHS2 UHS3_v1-2 UHS3_v1-3) # array of study names
#num=${#studies[@]}
#
## Get intersection set
#for ancestry in ${ancestry_list};do
#    bim_files=()
#    for (( i=0; i<${num}; i++ ));do
#        bim_files+=(${base_dir}/processing/${studies[$i]}/${ancestry}_filtered_mono.bim)
#    done
#    
#    echo -e "\nCalculating intersection between $ancestry ${study_list}...\n"
#    sort ${bim_files[@]} | uniq -dc | awk -v num=$num '$1 == num {print $3}' \
#        > ${base_dir}/processing/intersect/${ancestry}_variant_intersection.txt
#
#    # Make new PLINK binary file sets
#    for study in ${studies[@]}; do
#        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#            --noweb \
#            --bfile ${base_dir}/processing/${study}/${ancestry}_filtered_mono \
#            --extract ${base_dir}/processing/intersect/${ancestry}_variant_intersection.txt \
#            --make-bed \
#            --out ${base_dir}/processing/intersect/${study}_${ancestry}_filtered_snp_intersection
#    done
#done

```wc -l *txt

```

### Merge test
If merging multiple datasets together:

As a final check to confirm that our data sets are all compatible, a PLINK file set merge is conducted. If any issues persist then an error will be raised.

In [ ]:
#for ancestry in $ancestry_list;do
#
#    echo "Creating $ancestry merge-list"
#    touch ${base_dir}/processing/intersect/${ancestry}_merge_list.txt
#    for study in $study_list;do
#        echo ${base_dir}/processing/intersect/${study}_${ancestry}_filtered_snp_intersection >>\
#             ${base_dir}/processing/intersect/${ancestry}_merge_list.txt
#    done
#
## Merge file sets
#    echo -e "\n\n======== ${ancestry} ========\n\n"
#    /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#        --noweb \
#        --memory 4000 \
#        --merge-list ${base_dir}/processing/intersect/${ancestry}_merge_list.txt \
#        --make-bed \
#        --out ${base_dir}/processing/intersect/${ancestry}_studies_merged
#done

```
 wc -l *merged.bim
```

## Imputation preparation for Michigan Imputation Server
Visit the [MIS Getting Started Webpage](https://imputationserver.sph.umich.edu/start.html#!pages/help) for more information about the preparing the data for upload to MIS.


### VCF File Conversion

In [ ]:
### Split by chr and remove any individuals with missing data for whole chr

## if merged data sets together
#for ancestry in $ancestry_list;do
#    # Remove SNPs
#    /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#        --noweb \
#        --memory 4000 \
#        --bfile ${base_dir}/processing/intersect/${ancestry}_studies_merged \
#        --chr ${chr} \
#        --mind 0.99 \
#        --make-bed \
#        --out ${base_dir}/processing/impute_prep/${ancestry}_chr${chr}_for_phasing 
#done > ${base_dir}/processing/impute_prep/chr_splitting.log 

## if NO merging was done
for ancestry in $ancestry_list;do
    for chr in {1..23};do
        # Remove SNPs
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 4000 \
            --bfile ${base_dir}/processing/$study/${ancestry}_filtered_mono \
            --chr ${chr} \
            --mind 0.99 \
            --make-bed \
            --out ${base_dir}/processing/impute_prep/${ancestry}_chr${chr}_for_phasing 
    done
done > ${base_dir}/processing/impute_prep/chr_splitting.log 

__Note__: No subjects were removed.

```
grep remove *log
aa_chr10_for_phasing.log:0 people removed due to missing genotype data (--mind).
aa_chr11_for_phasing.log:0 people removed due to missing genotype data (--mind).
aa_chr12_for_phasing.log:0 people removed due to missing genotype data (--mind).
aa_chr13_for_phasing.log:0 people removed due to missing genotype data (--mind).
aa_chr14_for_phasing.log:0 people removed due to missing genotype data (--mind).
aa_chr15_for_phasing.log:0 people removed due to missing genotype data (--mind).
aa_chr16_for_phasing.log:0 people removed due to missing genotype data (--mind).
aa_chr17_for_phasing.log:0 people removed due to missing genotype data (--mind).

...

ea_chr5_for_phasing.log:0 people removed due to missing genotype data (--mind).
ea_chr6_for_phasing.log:0 people removed due to missing genotype data (--mind).
ea_chr7_for_phasing.log:0 people removed due to missing genotype data (--mind).
ea_chr8_for_phasing.log:0 people removed due to missing genotype data (--mind).
ea_chr9_for_phasing.log:0 people removed due to missing genotype data (--mind).
```

In [ ]:
# EC2 command line #

for chr in {1..23};do
    for ancestry in ${ancestry_list};do
        final_dir=${base_dir}/processing/impute_prep/${ancestry}
        mkdir $final_dir
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 5000 \
            --bfile ${final_dir}/../${ancestry}_chr${chr}_for_phasing \
            --output-chr M \
            --set-hh-missing \
            --recode vcf bgz \
            --out ${final_dir}/${ancestry}_chr${chr}_final
    done
done

Transfer the *.vcf.gz files to local machine (per chromosome) and then upload to MIS.

## Upload to Michigan Imputation Server (MIS)

### Uploading parameters
These are the parameters that were selected on MIS.

__Name__: UHS4_EA

__Reference Panel__ 1000G Phase 3 v5

__Input Files__ File Upload <br>

* Select Files - select VCF (.gz) files that were downloaded to local machine from cloud. <br>

__Phasing__: ShapeIT v2.r790 (unphased) 

__Population__: EUR

__Mode__: Quality Control & Imputation

* I will not attempt to re-identify or contact research participants.
* I will report any inadvertent data release, security breach or other data management incident of which I become aware.

# Download Imputed Results from MIS
First, download the data form the Michigan Imputation Server (MIS) by clicking on the link provided in the email they send out to alert you that your data has finished. Here you will find commands for downloading the data.

## EA
The zip files from Michigan Imputation Server (MIS) need to be inflated before you can begin working with them. They require a passcode that is sent by MIS to email.
* EA password: 

Then, inflate imputation results.

In [ ]:
cd /shared/data/studies/heroin/uhs4/genotype/imputed/20181209/ea
passW=

# QC-results
wget 

# SNP Statistics
wget

# Logs
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget
wget

# Imputation Results
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget 
wget
wget

/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name ea_impute_down \
    --script_prefix ea_impute_download \
    --mem 6.8 \
    --nslots 1 \
    --priority 0 \
    --program bash download_data_ea 

# inflate chr results
for file in *zip;do
    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ea_unzip \
        --script_prefix ea_unzip \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program unzip -P $passW $file
done


# we can remove the original imputed data from MIS after we inflate the zip files
rm -rf *zip

## AA
The zip files from Michigan Imputation Server (MIS) need to be inflated before you can begin working with them. They require a passcode that is sent by MIS to email.
* AA password: 

Then, inflate imputation results.

In [ ]:
cd /shared/data/studies/heroin/uhs4/genotype/imputed/20181209/aa
passW=

# QC-results

# SNP Statistics

# Logs

# Imputation Results

/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name aa_impute_down \
    --script_prefix aa_impute_download \
    --mem 6.8 \
    --nslots 1 \
    --priority 0 \
    --program bash download_data_aa 

# inflate chr results
for file in *zip;do
    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name aa_unzip \
        --script_prefix aa_unzip \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program unzip -P $passW $file
done
# inflate chr results
/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name aa_unzip \
    --script_prefix aa_unzip \
    --mem 6.8 \
    --nslots 1 \
    --priority 0 \
    --program unzip -P $passW *zip 
# we can remove the original imputed data from MIS after we inflate the zip files
rm -rf *zip

# Upload to S3

In [ ]:
cd /shared/data/studies/heroin/uhs4/genotype/imputed/20181209

for ancestry in {aa,ea};do
    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${ancestry}_upload \
        --script_prefix ${ancestry}_upload \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program aws s3 sync ${ancestry} s3://rti-heroin/uhs4/data/genotype/imputed/20181209/${ancestry}/
done